#### HMA1 Class 
This is a demo notebook to discover functionalities of the HMA1 class for relational data modelling.

#### What is HMA1?
The bulian.relational.HMA1 class implements a Hierarchical Modeling Algorithm which is an algorithm that allows to recursively walk through a relational dataset and applies tabular models across all the tables in a way that lets the models learn how all the fields from all the tables are related.

#### Import all the relevant packages

In [1]:
import os,sys
import pandas as pd
from bulian.metrics.demo import sample_relational_demo
from bulian.metrics.reports import get_multi_table_report
from bulian.utils import display_tables
from bulian.relational import HMA1

: 

: 

#### Start by Loading Bulian Demo relational dataset

In [ ]:
metadata, tables = sample_relational_demo()

In [ ]:
metadata

##### This returns two objects, the relational data tables and the metadata associated with each table 

In [ ]:
print(metadata)

In [ ]:
for name, table in tables.items():
    print(name, table.shape)

In [ ]:
display_tables(tables)

#### Deep dive into metadata class

- Get Tables to view associated table names
- Get parents(table) to view child-parent releationship for a particular table
 

In [ ]:
metadata.get_tables()

In [ ]:
metadata.get_parents('sessions')

#### Next, lets fit a HMA1 model to learn this data to eventually sample synthetic data rows

- Import bulian.relational.HMA1 and create an instance of it passing relevant metadata
- Fit the instance to the tables dict

In [ ]:
model = HMA1(metadata)
model.fit(tables)

*The fit method basically iterates through all the datasets in the tables dict and models each dataset using Gaussian Copula models*  

#### Next, sample synthetic data rows from the fitted HMA1 model 

In [ ]:
new_data = model.sample(num_rows=100,)

In [ ]:
display_tables(new_data)

In [ ]:
for name, table in new_data.items():
    print(name, table.shape)

#### Generate Report on Real and Synthetic Data

After new data has been generated you can create a quality report on the same. The function takes in the real data, synthetic data and metadata as required input parameters. You can also specific the numeric and discrete columns explicitly or let the function determine it accordingly. 

The inputs for numeric and discrete columns is dictionary with the key name being the table name and the values is a list of valid column names present in the tables.

##### Generate Report without specifying columns

In [ ]:
get_multi_table_report(tables, new_data, metadata)

##### Generate Report with defined columns

As mentioned above you can also specify the numeric and discrete columns and pass them as parameters while generating the report.

In [ ]:
discrete_columns = {
    'users': ['country', 'gender'],
    'sessions': ['device', 'os'],
    'transactions': ['timestamp', 'cancelled']
}

numeric_columns = {
    'users': ['age'],
    'sessions': ['minutes'],
    'transactions': ['amount']
}

In [ ]:
get_multi_table_report(tables, new_data, metadata, discrete_features=discrete_columns, numeric_features=numeric_columns)

##### Generate Report on a Dashboard

Reports can be viewed on a dashboard webpage by using the option boolean <b>show_dashboard</b> parameter on the <b>get_mutli_table_report</b> function. The dashboard is a Dash application which runs on a local server. You can also specify which port to use for the local server, by default the app would run on <b>8050</b>.

In [ ]:
get_multi_table_report(tables, new_data, metadata, discrete_features=discrete_columns, numeric_features=numeric_columns, show_dashboard=True, port=8050)

#### Save and share the model:  
Once you have fitted the model, all you need to do is call its save method passing the name of the file in which you want to save the model. Note that the extension of the filename is not relevant, but we will be using the .pkl extension to highlight that the serialization protocol used is pickle.

In [ ]:
model.save('relational_model.pkl')

**Important** If you inspect the generated file you will notice that its size is much smaller than the size of the data that you used to generate it. This is because the serialized model contains **no information about the original data**, other than the parameters it needs to generate synthetic versions of it. This means that you can safely share this `relational_model.pkl` file without the risk of disclosing any of your real data!

#### Sample only a subset of tables? 

In some occasions you will not be interested in generating rows for the entire dataset and would rather generate data for only one table and its children.
To do this you can simply pass the name of the table that you want to sample.
For example, pass the name sessions to the sample method, the model will only generate data for the sessions table and its child table, transactions.

In [ ]:
model.sample('sessions', num_rows=25)

If you want to further restrict the sampling process to only one table and also skip its child tables, you can add the argument sample_children=False. For example, you can sample data from the table users only without producing any rows for the tables sessions and transactions.

In [ ]:
model.sample('users', num_rows=5, sample_children=False)

**Note** In this case, since we are only producing a single table, the output is given directly as a `pandas.DataFrame` instead of a dictionary.

### Fin ###